<a href="https://colab.research.google.com/github/ChrisM2309/LangchainTesting/blob/main/FinanceBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain

In [5]:
pip install langchain_community openai

In [3]:
!pip install openai

Haciendo los import y configurando la api key

In [78]:
import os
from openai import OpenAI as OpenAI

from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool, load_tools
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.llms import OpenAI as OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate

import json

from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

client = OpenAI(
    api_key = OPENAI_API_KEY
)
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [11]:
#LLM TYPE
llm = OpenAI(temperature = 0.5)
llm_text = "Say hello world in python"
print(llm.predict(llm_text))

<ipython-input-11-602f8cb4f282>:4: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm.predict(llm_text))




print("Hello World")


In [14]:
#ChatBot Type
chat = ChatOpenAI(model="gpt-4o-mini")
#chat_text = "Dame las entidades y relaciones para una base de datos de una veterinaria" #Aqui van las preguntas
#messages = [HumanMessage(content=chat_text)]
#print(chat.predict_messages(messages).content)

<ipython-input-14-84c2753dc516>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(model="gpt-4o-mini")


**Simular escenarios - PROMPT TEMPLATE**

In [23]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages_def = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(  # Usa chat.completions para modelos de chat
        model=model,
        messages=messages_def,
        temperature=0
    )
    return response.choices[0].message.content


ingresos = f"""- 25/09/2023: $1,200 (Ventas)
  - 27/09/2023: $800 (Ventas)
  """
gastos = f"""  - 25/09/2023: $300 (Insumos)
  - 26/09/2023: $150 (Servicios)
  - 28/09/2023: $500 (Salarios)
  """
language = f"""Ingles"""

prompt_edit = f"""
  Datos financieros de una PYME:
- Ingresos: {ingresos}
- Gastos: {gastos}

Instrucciones:
1. Calcula el total de ingresos, gastos y saldo (ingresos - gastos).
2. Identifica la categoría con mayor gasto.
3. Sugiere una acción para mejorar el flujo de caja basado en los datos.
4. Explica los resultados de manera clara y sencilla para un dueño de PYME sin conocimientos financieros avanzados.
  """

rewrote = get_completion(prompt = prompt_edit)
print(rewrote)

AttributeError: 'OpenAI' object has no attribute 'chat'

# CREAR TOOLS


In [91]:
def registrar_transaccion(input_str):
    # Lógica para guardar en una base de datos (puedes usar SQLite o una lista temporal)
    params = json.loads(input_str)
    tipo = params["tipo"]
    monto = params["monto"]
    categoria = params["categoria"]
    descripcion = params["descripcion"]
    return f"✅ {tipo} de ${monto} en {categoria} registrado."


#--- PROBANDO CON CHAINS
# Crear una cadena (Chain) para registrar transacciones
prompt_registrar = PromptTemplate(
    input_variables=["input"],
    template=""""
    Del input recibido, analiza con precaucion y extrae los parametros necesarios para hacer el registro:
    - Tipo (Ingreso o Gasto)
    - Monto (número)
    - Categoría (texto)
    - Descripción (texto, opcional, usa '' si no está presente)

    Texto: {texto}

    Devuelve el resultado en formato: ✅ {tipo} de ${monto} en {categoria} registrado.""
    """
)

chain_registrar = LLMChain(llm=chat, prompt=prompt_registrar)

#--- CONTINUACION

tool_registrar = Tool(
    name="registrar_transaccion",
    func= registrar_transaccion,
    description="Registra un ingreso o gasto con tipo (Ingreso/Gasto), monto (número), categoria (texto) y descripcion (texto).")


def calcular_flujo_caja(ingresos, gastos):
    saldo = ingresos - gastos
    return f"🔹 Ingresos: ${ingresos} 🔹 Gastos: ${gastos} 🔹 Saldo: ${saldo}"

tool_flujo_caja = Tool(
    name="calcular_flujo_caja",
    func=calcular_flujo_caja,
    description="Calcula el flujo de caja."
)

def configurar_presupuesto(categoria, monto):
    # Lógica para guardar el presupuesto
    return f"✅ Presupuesto de ${monto} para {categoria} configurado."

tool_presupuesto = Tool(
    name="configurar_presupuesto",
    func=configurar_presupuesto,
    description="Configura un presupuesto para una categoría."
)

def responder_pregunta_financiera(pregunta):
    # Usar el modelo de lenguaje para responder preguntas generales
    respuesta = llm(pregunta)
    return respuesta

tool_preguntas = Tool(
    name="responder_pregunta",
    func=responder_pregunta_financiera,
    description="Responde preguntas financieras generales."
)

# CONFIGURAR AL AGENTE

In [95]:
memory = ConversationBufferMemory(memory_key="chat_history")

# Inicializar el agente
# Lista de herramientas
tools = [tool_registrar,tool_preguntas]

# Inicializar el agente
agente = initialize_agent(
    tools = tools,
    llm = chat,
    agent="conversational-react-description",
    verbose=True,
    max_iterations = 3,
    memory = memory
    )

TESTEO DEL AGENTE

In [96]:
respuesta = agente.run("Registra un gasto de $150 en insumos.")
respuesta = agente.run("Registra un gasto de $200 en publiicidad.")
respuesta = agente.run("Cuanto he gastado en total?")

print(respuesta)
# Salida: ✅ Gasto de $150 en insumos registrado.



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: registrar_transaccion
Action Input: {"tipo":"Gasto","monto":150,"categoria":"Insumos","descripcion":"Gasto en insumos"}
Observation: ✅ Gasto de $150 en Insumos registrado.
Thought:Do I need to use a tool? No  
AI: El gasto de $150 en insumos ha sido registrado correctamente. Si necesitas ayuda con algo más, no dudes en decírmelo.

> Finished chain.


> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: registrar_transaccion
Action Input: {"tipo": "Gasto", "monto": 200, "categoria": "Publicidad", "descripcion": "Gasto en publicidad"}
Observation: ✅ Gasto de $200 en Publicidad registrado.
Thought:Do I need to use a tool? No  
AI: El gasto de $200 en publicidad ha sido registrado correctamente. Si necesitas ayuda con algo más, no dudes en decírmelo.

> Finished chain.


> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No
AI: Hasta ahora, h